In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field

# Model

In [2]:
class Person(BaseModel):
    """Information about a person."""

    name: Optional[str] = Field(default=None, description="The name of the person.")
    birth_date: Optional[str] = Field(default=None, description="The person's birth date")


class Model(BaseModel):
    
    persons: Optional[List[Person]]

# LangChain

In [32]:
# Define the parser
parser = PydanticOutputParser(pydantic_object=Model)

# Define prompt
prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer the user query. Wrap the output in ```json and ``` tags\n{format_instructions}. Do not provide more fields that the one required."),
        ("human", "{text}")
    ]).partial(format_instructions=parser.get_format_instructions())

# Define the LLM
llm = ChatOllama(model="llama3.1", temperature=0)

# Build the chain
chain = prompt | llm | parser


## Extract information

In [41]:
# initial_text =  "Emanuel Burckhardt est né le 25.11.1744 à Bâle, et mort le 21.5.1820 à Palerme, protestant. Her husband is named Aurelius and was born on 1st September 1999."
initial_text = "Emanuel Burckhardt naît le 25.11.1744 à Bâle, meurt le 21.5.1820 à Palerme, protestant, de Bâle. " \
               "Fils de John Doe, lieutenant au service de France. " \
               "Marié(e) (1783) à Theresia Münster, de Bâle. " \
               "Alfred Abion was born on 1st September 2021."
               #"Au service de France jusqu'en 1787, puis de Naples. " \
               #"Commandant en chef des troupes du royaume de Sicile (1802), capitaine général des troupes du royaume des Deux-Siciles (1815). " \
               #"Emanuel Burckhardt dut sa carrière à ses compétences et à sa loyauté envers Ferdinand IV."

chain.invoke({"text": initial_text})

Model(persons=[Person(name='Emanuel Burckhardt', birth_date='25.11.1744'), Person(name='Alfred Abion', birth_date='1st September 2021')])

In [40]:
def analyze_chain(text):

    print('=== INITIAL TEXT ===')
    print(text)

    prompt_result = prompt.invoke({"text": initial_text})
    print('=== PROMPT ===')
    for message in prompt_result.messages:
        print(message.content)
        print('---')

    llm_result = llm.invoke(prompt_result)
    print('=== LLM ANSWER ===')
    print(llm_result.content)

    parser_result = parser.invoke(llm_result)
    print('=== PARSED ANSWER ===')
    print(parser_result)


analyze_chain(initial_text)

=== INITIAL TEXT ===
Emanuel Burckhardt naît le 25.11.1744 à Bâle, meurt le 21.5.1820 à Palerme, protestant, de Bâle. Fils de John Doe, lieutenant au service de France. Marié(e) (1783) à Theresia Münster, de Bâle. 
=== PROMPT ===
Answer the user query. Wrap the output in ```json and ``` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"persons": {"title": "Persons", "type": "array", "items": {"$ref": "#/definitions/Person"}}}, "definitions": {"Person": {"title": "Person", "description": "Information about a person.", "type": "object", "properties": {"name": {"title": "Na

___

In [14]:
from langchain_ollama import ChatOllama

# Define the parser
parser = PydanticOutputParser(pydantic_object=Model)

# Define the LLM
llm = ChatOllama(model="llama3.1", temperature=0)

# Define the prompt template
system_message = "Answer the user query. Wrap the output in ```json and ``` tags\n{format_instructions}"
human_message = "{text}"

def run_chain(text):
    # Manually create the formatted prompt
    format_instructions = parser.get_format_instructions()
    full_prompt = system_message.format(format_instructions=format_instructions) + "\n" + human_message.format(text=text)
    
    print(f"Input to LLM:\n{full_prompt}\n")
    
    # Get the LLM response
    llm_response = llm.invoke(full_prompt)  # Directly call the LLM
    print(f"Raw LLM Response:\n{llm_response}\n")
    
    # Parse the response
    parsed_response = parser.parse(llm_response)
    print(f"Parsed Response:\n{parsed_response}\n")
    
    return parsed_response

# Example usage
run_chain(initial_text)

Input to LLM:
Answer the user query. Wrap the output in ```json and ``` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"persons": {"title": "Persons", "type": "array", "items": {"$ref": "#/definitions/Person"}}}, "definitions": {"Person": {"title": "Person", "description": "Information about a person.", "type": "object", "properties": {"name": {"title": "Name", "description": "The name of the person.", "type": "string"}, "birth_date": {"title": "Birth Date", "description": "The person's birth date", "type": "string"}}}}}
```
Emanuel Burckhardt est né le 25.11.1744 à Bâ

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [ ]:
text = """Morgane was born in Mulhouse in 1998, and Gaétan in Sierentz in 1888."""
chain.invoke({"text": text})

In [ ]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]

llm = ChatOllama(model="llama3.1", temperature=0)

# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in ```json and `` tags\n{format_instructions}"
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

